In [1]:
# default_exp ipfsspec

In [2]:
#hide
%load_ext autoreload
%autoreload 2

In [3]:
# import sys
# sys.path.append("../")

# IPFSSpec

> Read and Write implementation FSSpec for IPFS 

In [4]:
#hide
from nbdev.showdoc import *

In [5]:
#export
from os.path import isdir
from typing import Union, List
import json
import random
import time
import requests
import pandas as pd
import imageio
from io import StringIO,BytesIO
from requests.exceptions import HTTPError
from fsspec.spec import AbstractFileSystem
from ipfsspec.core import IPFSBufferedFile
from ipfshttpclient.multipart import stream_files, stream_directory

In [6]:
#export 
from ipfspy.utils import GATEWAYS_API_READ, GATEWAYS_API_WRITE, parse_error_message, parse_response, get_coreurl

## IPFSGateway

In [ ]:
#export
class IPFSGateway:
    def __init__(self, 
        local:bool=True, # Use local IPFS deamon or not
        coreurl:str=None, # Core URL of an alterative gateways to use 
    ):
        'Starts a IPFS Gateway either using local node or infura. If given `coreurl`, will use that for the gateway'
        self.coreurl = coreurl
        if self.coreurl is None:
            self.url = self.get_gateway(local=local)
            
        else:
            self.url = self.get_gateway(coreurl=coreurl)
            
        self.session = requests.Session()
        adapter = requests.adapters.HTTPAdapter(pool_connections=100, pool_maxsize=100)
        self.session.mount('http://', adapter)
        self.session.mount('https://', adapter)
    
    def get_gateway(self,
        local:bool=True, # If local uses local node, else uses Infura.io gateway
        coreurl:str=None, # URL of other gateways
    ):
        'Set the core url for convenience'
        
        return get_coreurl(local=local, coreurl=coreurl)

        
    def get(self, 
        cid:str, # Path to the IPFS object
        **kwargs
    ):
        'Get a file/directory from IPFS'
        
        params = {}
        params['arg'] = cid
        params.update(kwargs)
        
        res = self.session.post(f'{self.url}/get', params=params)
            
        if res.status_code == 200:
            return res, parse_response(res)
        
        else:
            raise HTTPError (parse_error_message(res))
            
    def cat(self, 
        cid:str, # Path to the IPFS object
        **kwargs
    ):
        'Read a file from IPFS'
        
        params = {}
        params['arg'] = cid
        params.update(kwargs)
        
        res = self.session.post(f'{self.url}/cat', params=params)
        
        if res.status_code == 200:
            return res, res.text
        
        else:
            if res.status_code == 500:
                raise TypeError (f"dag node {path} is a directory; Provide a file CID")
            else:
                raise HTTPError (parse_error_message(res))

    def apipost(self, 
        call:str, # The call type to post e.g. 'add', 'ls', 'pin/add', 'pin/ls'
        filepath:Union[str, List[str]]=None, # Path to files or directory or IPFS Path
        directory:bool=False, # Is filepath a directory 
        chunk_size=200000, # Chunk size to use
        **kwargs): 
        'Makes `post` call to the HTTP APPI'
    
        
        if call == 'add':
            if isdir(filepath):
                if directory == False:
                    raise TypeError (f"{filepath} is a directory. Set arg directory as True")
            
            params = {}
            params.update(kwargs)
            
            if not directory:
                data, headers = stream_files(filepath, chunk_size=chunk_size)

            else:
                data, headers = stream_directory(filepath, chunk_size=chunk_size)
            
            res = self.session.post(f'{self.url}/add',
                                     params=params,
                                     data=data,
                                     headers=headers)
            
            if res.status_code == 200:
                return res, parse_response(res)
            
            else:
                raise HTTPError (parse_error_message(res))
                
        else:
            params = {}
            params['arg'] = filepath
            params.update(kwargs)
            
            res = self.session.post(f'{self.url}/{call}', params=params)
                
            if res.status_code == 200:
                return res, parse_response(res)
            
            else:
                raise HTTPError (parse_error_message(res))

    def head(self, 
        cid:str, # Path to the IPFS object
        headers=None, 
        **kwargs
    ):     
        
        res,_ = self.get(cid)
        
        return res.headers

### Using IPFSGateway

Using local-node supports all function. Infura gateway supports both read and write but not the complete set as offered by the local-node. We have a list of read and read/write gateways. You can access them as such:

In [ ]:
#ignoretest
GATEWAYS_API_READ

In [ ]:
#ignoretest
GATEWAYS_API_WRITE

In this doc, we will be using the local-node unless specified. 

Extending a client session using the local node.

In [ ]:
#ignoretest
gw = IPFSGateway(local=True); gw.url

Extending a client session using infura.

In [ ]:
#ignoretest
gw = IPFSGateway(local=False); gw.url

Extending a client session using a public gateway.

In [ ]:
#ignoretest
gw = IPFSGateway(coreurl=random.choice(GATEWAYS_API_READ)); gw.url

In [ ]:
#ignoretest
show_doc(IPFSGateway.get)

Let's use a local-node to get a IPFS file

In [ ]:
#ignoretest
gw = IPFSGateway(local=True)

In [ ]:
#ignoretest
res, cont = gw.get('QmWAQjxm6CKaAjHjwPSqWmN8RMecuHS1bxEufqCNVSq96e'); cont[500:700]

Use the `cat` function which return the file without the headers and other stuffs that the `get` method returns.

In [ ]:
#ignoretest
show_doc(IPFSGateway.cat)

In [ ]:
#ignoretest
res, cont = gw.cat('QmWAQjxm6CKaAjHjwPSqWmN8RMecuHS1bxEufqCNVSq96e'); cont[:5]

Like previously mentioned, infura supports read and write but not the complete set of functions that comes with the local node. Let's see an example. Here, the `ls` call lists the content of a directory. Let's try the same thing with `infura`

In [ ]:
#ignoretest
r, res = gw.apipost(call='ls', filepath='QmWAQjxm6CKaAjHjwPSqWmN8RMecuHS1bxEufqCNVSq96e'); len(res[0])

In [ ]:
#ignoretest
gw = IPFSGateway(local=False)
r, res = gw.apipost(call='ls', filepath='QmWAQjxm6CKaAjHjwPSqWmN8RMecuHS1bxEufqCNVSq96e'); len(res[0])

As can be inferred from the error message, `ls` call is not supported by inura ar this point. 

## IPFSFileSystem (fsspec-like)

In [ ]:
#export
class IPFSFileSystem(AbstractFileSystem):
    protocol = "ipfs"
    def __init__(self, 
        local=True, # Use local IPFS deamon or not
        coreurl:str=None, # Core URL of an alterative gateways to use 
        **kwargs):
        'fsspec like read/write IPFS filesystem'
        
        super(IPFSFileSystem, self).__init__(local, **kwargs)
        
        if coreurl is None:
            self.gw = IPFSGateway(local=local)
        
        else:
            self.gw = IPFSGateway(coreurl)
    
    def ls(self, 
        cid:str, # Path of the IPFS object
        detail=True, # Verbose
        **kwargs):
        'List the links of a IPFS file/directory'
        
        _, res = self.gw.apipost("ls", arg=cid)
        
        links = res[0]["Objects"][0]["Links"]
        types = {1: "directory", 2: "file"}
        
        if detail:
            return [{"name": cid + "/" + link["Name"],
                     "size": link["Size"],
                     "type": types[link["Type"]]}
                    for link in links]
        
        else:
            return [cid + "/" + link["Name"]
                    for link in links]
        
    def cat_file(self, 
        cid:str, # Path of the IPFS object
    ):        
        
        r, data = self.gw.cat(cid)      
        
        return r.content
                
    def _open(
        self,
        cid, # Path of the IPFS object
        mode="rb",
        block_size=None,
        autocommit=True,
        cache_options=None,
        **kwargs
    ):
        'Return raw bytes-mode file-like from the file-system'
        
        return IPFSBufferedFile(
            self,
            cid,
            mode,
            block_size,
            autocommit,
            cache_options=cache_options,
            **kwargs
        )
    
    def info(self, 
        cid, # Path of the IPFS object
        **kwargs):
        
        path = self._strip_protocol(cid)

        headers = {"Accept-Encoding": "identity"}  # this ensures correct file size
        response_headers = self.gw.head(cid, headers)

        info = {"name": cid}
        if "X-Content-Length" in response_headers:
            info["size"] = int(response_headers["X-Content-Length"])
        elif "X-Content-Range" in response_headers:
            info["size"] = int(response_headers["X-Content-Range"].split("/")[1])

        if "ETag" in response_headers:
            etag = response_headers["ETag"].strip("\"")
            info["ETag"] = etag
            if etag.startswith("DirIndex"):
                info["type"] = "directory"
                info["CID"] = etag.split("-")[-1]
            else:
                info["type"] = "file"
                info["CID"] = etag
        return info
    
    def write(self,
        filepath, # Path to file/files/directories to write to IPFS
        directory=False, # Is filepath a directory
        chunk_size=200000, # Chunk size to use
        **kwargs
    ):
        'Write the given file/files/directories to the IPFS network'
        
        return self.gw.apipost('add', filepath=filepath, directory=directory, chunk_size=chunk_size, **kwargs)
    
    def read_csv(self, 
        cid:str,
        delimeter:str=','
    ):
        r, data = self.gw.cat(cid)      
        
        return pd.read_csv(StringIO(data), delimiter=delimeter)
    
    def read_json(self, 
        cid:str, 
    ):
        r, data = self.gw.cat(cid)      
        
        return pd.read_json(StringIO(data))
    
    def read_image(self,
        cid:str,
    ):
        data = self.cat_file(cid)
        
        return imageio.v2.imread(BytesIO(data))

### Using IPFSSpec

In [ ]:
#ignoretest
fs = IPFSFileSystem(local=True)

call the `ls` method to list the directories

In [ ]:
#ignoretest
res = fs.ls('QmZnxARhJWsCbTxiAzoRhnxHgMtoEkNJNS8DGLCBEMvm4V'); res

Use the `open` and `read` on a IPFS file

In [ ]:
#ignoretest
fs.open('QmWAQjxm6CKaAjHjwPSqWmN8RMecuHS1bxEufqCNVSq96e').read()[:10]

`Write` a file, files or a directory to IPFS

In [ ]:
#ignoretest
fs.write('output', directory=True)

Read a csv straight from IPFS into a `pandas` object

In [ ]:
#ignoretest
df = fs.read_csv('QmZnxARhJWsCbTxiAzoRhnxHgMtoEkNJNS8DGLCBEMvm4V'); df.head()

Similarly, read a json straight from IPFS into a `pandas` object

In [ ]:
#ignoretest
df = fs.read_json('QmUUSHH2ycqciruPaRcptpUQDDeRiMV3G9PxA6KUupXTJV'); df.head()

In [ ]:
#ignoretest
from matplotlib import pyplot as plt

img = fs.read_image("bafybeieyrzfz46sfmagpwerlzrutr3didbu3nii3oiwk2tbc7rze25m52a")
plt.figure(figsize=(10,7))
plt.imshow(img, interpolation='nearest')
plt.axis('off')
plt.show()
